<a href="https://colab.research.google.com/github/AkashSalmuthe/Python-Noteboks/blob/main/Missing_Value_Imputation_Using_SKlearn_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv('/content/drive/MyDrive/AI Production/5. Data Preprocessing and Feature Engineering/house_train.csv')
test = pd.read_csv('/content/drive/MyDrive/AI Production/5. Data Preprocessing and Feature Engineering/house_test.csv')

print('Shape Of Train Dataset: ',train.shape)
print('Shape Of Test Dataset: ',test.shape)

Shape Of Train Dataset:  (1460, 81)
Shape Of Test Dataset:  (1459, 80)


In [3]:
X_train = train.drop(columns='SalePrice') # Input 
y_train = train['SalePrice'] # Output only
X_test = test.copy()
print("Shape of X_train: ",X_train.shape)
print("Shape of y_train: ",y_train.shape)
print("Shape of X_test: ",X_test.shape)

Shape of X_train:  (1460, 80)
Shape of y_train:  (1460,)
Shape of X_test:  (1459, 80)


# **Missing Value Imputation**

In [4]:
isnull_sum = X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [6]:
num_vars = X_train.select_dtypes(include=['int64','float64']).columns
num_vars_miss = [var for var in num_vars if isnull_sum[var]>0]
num_vars_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [7]:
cat_vars = X_train.select_dtypes(include=['object']).columns
cat_vars_miss = [var for var in cat_vars if isnull_sum[var]>0]
cat_vars_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [10]:
num_var_mean = ['LotFrontage']
num_var_median = ['MasVnrArea', 'GarageYrBlt']
cat_var_miss = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','FireplaceQu']
cat_var_misiing = ['GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [17]:
num_var_mean_imputer = Pipeline(steps=[("imputer",SimpleImputer(strategy='mean'))])
num_var_median_imputer = Pipeline(steps=[("imputer",SimpleImputer(strategy='median'))])
cat_var_mode_imputer = Pipeline(steps=[("imputer",SimpleImputer(strategy='most_frequent'))])
num_var_missing_imputer = Pipeline(steps=[("imputer",SimpleImputer(strategy='constant',fill_value='missing'))])

In [18]:
preprocessor = ColumnTransformer(transformers=[('mean_imputer',num_var_mean_imputer,num_var_mean),
                                ('median_imputer',num_var_median_imputer,num_var_median),
                                ('mode_imputer',cat_var_mode_imputer,cat_var_miss),
                                ('missing_imputer',num_var_missing_imputer,cat_var_misiing)])

In [19]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_imputer',
                                 Pipe

In [20]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_im

In [23]:
preprocessor.named_transformers_['mean_imputer'].named_steps['imputer'].statistics_

array([70.04995837])

In [27]:
train['LotFrontage'].mean()  

70.04995836802665

In [28]:
preprocessor.named_transformers_['mode_imputer'].named_steps['imputer'].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [31]:
X_train_clean = preprocessor.transform(X_train)
X_test_clean = preprocessor.transform(X_test)

In [32]:
X_train_clean

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [33]:
X_test_clean

array([[80.0, 0.0, 1961.0, ..., 'missing', 'MnPrv', 'missing'],
       [81.0, 108.0, 1958.0, ..., 'missing', 'missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'missing', 'MnPrv', 'missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'missing', 'missing', 'missing'],
       [62.0, 0.0, 1980.0, ..., 'missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [34]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('missing_imputer', Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]), ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,


In [36]:
X_train_clean_miss = pd.DataFrame(X_train_clean,columns=num_var_mean+num_var_median+cat_var_miss+cat_var_misiing)
X_train_clean_miss.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,missing,missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,missing,missing,missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing


In [37]:
X_train_clean_miss.isnull().sum().sum()

0